<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=13dafb9fc76cf7fc35bc3ac65c18985f1c202bd9754d20b6ecd9dc91de7c5c3c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 09:53:48
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: 219.00 (0.0%)
Current PnL: -22.17 L (-14.53%)
CY Booked + Current PnL: -8.28 L (-5.43%)
-------------------
Total profit:  1.65 L
Total loss:  -23.82 L
-------------------
Total Booked + Current PnL: 18.70 L (14.89%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.67%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.58 L (60.98%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.14,-13.99,16.38,0.09,14198.0,-14101.0,86676.0,101.28,51.0,M-SC,3.55,253.0,-0.99,0.60,13.87,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.65,-9.64,10.69,0.02,16861.0,-16831.0,157728.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.20,OX40N,NTT,PAINTS
52,MEDANTA,1486.00,0.17,5.66,17.43,24.08,23379.0,7182.0,134132.0,-2.92,51.0,X-SC,5.52,89.0,0.31,0.93,26.38,XY24,NTT,HEALTHCARE
83,VOLTAS,1530.00,-0.97,7.45,11.39,19.69,23466.0,14283.0,206025.0,-3.01,50.0,X-MC,3.34,78.0,0.61,1.43,14.75,XY25,NTT,AC
35,ICICIGI,2252.93,-0.61,6.60,12.75,20.19,23692.0,11503.0,185823.0,-16.49,49.0,X-MC,7.37,68.0,0.49,1.29,22.71,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.1,3.29,-24.98,86.09,39.61,66203.0,-25600.0,76900.0,-49.31,39.0,H-SC,20.49,135.0,-0.39,0.54,8.39,AR,ATH,MISC
7,ATULAUTO,844.0,2.59,-18.45,78.51,45.57,126206.0,-36380.0,160752.0,3478.38,52.0,M-SC,4.02,244.0,-0.29,1.12,13.97,XY24,NTT,AUTO
24,EASEMYTRIP,26.4,2.10,-59.82,262.64,45.70,186574.0,-105777.0,71038.0,-26.33,29.0,M-SC,23.00,205.0,-0.57,0.49,2.10,XY24,NTT,TRAVEL
67,SIS,528.0,1.92,-21.39,54.25,21.26,47353.0,-23746.0,87286.0,2062.69,59.0,H-SC,3.81,166.0,-0.50,0.61,17.93,OX40N,NTT,MISC
65,SHALBY,327.0,1.62,-17.37,51.40,25.10,76936.0,-31462.0,149681.0,1000.48,37.0,M-SC,15.96,250.0,-0.41,1.04,19.95,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-11.12,-14.46,112.67,81.92,110643.0,-16597.0,98201.0,-49.86,23.0,M-SC,3.33,236.0,-0.15,0.68,16.25,XR,NTT,DURABLES
39,INDIGOPNTS,1408.0,-2.65,-9.64,10.69,0.02,16861.0,-16831.0,157728.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.20,OX40N,NTT,PAINTS
37,IEX,219.0,-1.19,-4.94,56.35,48.63,107107.0,-9879.0,190075.0,-36.71,50.0,H-SC,15.64,136.0,-0.09,1.32,6.04,XR,NTT,MISC
64,SFL,1287.0,-1.02,-40.32,112.17,26.63,175549.0,-105715.0,156503.0,18.50,34.0,M-SC,5.45,239.0,-0.60,1.09,2.28,XY24,NTT,MISC
83,VOLTAS,1530.0,-0.97,7.45,11.39,19.69,23466.0,14283.0,206025.0,-3.01,50.0,X-MC,3.34,78.0,0.61,1.43,14.75,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.44,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,38.0,H-SC,3.11,139.0,0.00,0.86,23.49,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.05,-0.77,36.68,35.63,76280.0,-1613.0,207961.0,-14.97,43.0,H-MC,3.98,119.0,-0.02,1.45,15.77,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.23,-4.42,117.52,107.91,165844.0,-6524.0,141120.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,0.91,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.65,-9.64,10.69,0.02,16861.0,-16831.0,157728.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.20,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.14,-13.99,16.38,0.09,14198.0,-14101.0,86676.0,101.28,51.0,M-SC,3.55,253.0,-0.99,0.60,13.87,OX40N,NTT,DURABLES
67,SIS,528.00,1.92,-21.39,54.25,21.26,47353.0,-23746.0,87286.0,2062.69,59.0,H-SC,3.81,166.0,-0.50,0.61,17.93,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.06,-21.95,45.39,13.47,95534.0,-59193.0,210474.0,-67.92,33.0,H-SC,1.50,158.0,-0.62,1.47,7.02,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.19,2.42,68.20,72.27,118360.0,4100.0,173548.0,-11.98,59.0,M-LC,3.86,99.0,0.03,1.21,8.74,XR,NTT,IT
38,INDIAMART,4810.62,-0.44,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,38.0,H-SC,3.11,139.0,0.00,0.86,23.49,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.19,2.42,68.20,72.27,118360.0,4100.0,173548.0,-11.98,59.0,M-LC,3.86,99.0,0.03,1.21,8.74,XR,NTT,IT
38,INDIAMART,4810.62,-0.44,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,38.0,H-SC,3.11,139.0,0.00,0.86,23.49,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.05,-0.77,36.68,35.63,76280.0,-1613.0,207961.0,-14.97,43.0,H-MC,3.98,119.0,-0.02,1.45,15.77,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.23,-4.42,117.52,107.91,165844.0,-6524.0,141120.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,0.91,OX40N,ATH,CABLES
37,IEX,219.00,-1.19,-4.94,56.35,48.63,107107.0,-9879.0,190075.0,-36.71,50.0,H-SC,15.64,136.0,-0.09,1.32,6.04,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.18,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,25.0,X-SC,13.19,92.0,-0.50,0.58,0.18,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.43,-15.24,67.74,42.18,158711.0,-42116.0,234294.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.49,XY24,NTT,AUTO
27,GLAXO,3466.20,-0.53,-0.60,39.99,39.15,79212.0,-1204.0,198080.0,-20.20,33.0,X-MC,10.09,60.0,-0.02,1.38,28.22,X40,ATH,PHARMA
55,PGHH,17907.65,0.21,-4.37,39.87,33.76,76568.0,-8775.0,192045.0,-32.51,34.0,X-MC,3.72,57.0,-0.11,1.34,0.60,X40,ATH,FMCG
46,JSWINFRA,342.00,1.58,-7.69,24.82,15.23,45905.0,-15397.0,184950.0,-21.02,36.0,X-MC,6.88,66.0,-0.34,1.29,22.09,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.09,-1.62,12.26,10.44,24139.0,-3242.0,196896.0,-41.15,43.0,X-LC,1.02,5.0,-0.13,1.37,3.85,X40,NTT,FMCG
3,ACC,3906.0,-0.66,-21.33,108.65,64.15,203393.0,-50751.0,187200.0,-54.13,54.0,X-SC,1.48,82.0,-0.25,1.30,5.01,XY24,BTT,CEMENT
76,TMPV,600.0,-0.43,-15.24,67.74,42.18,158711.0,-42116.0,234294.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.49,XY24,NTT,AUTO
36,ICICIPRULI,790.0,0.51,3.62,26.39,30.97,48331.0,6402.0,183140.0,-20.80,58.0,X-MC,1.83,75.0,0.13,1.28,16.64,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.87,-11.60,41.99,25.51,69074.0,-21595.0,164500.0,0.93,63.0,H-LC,1.88,49.0,-0.31,1.15,21.22,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.18,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,25.0,X-SC,13.19,92.0,-0.50,0.58,0.18,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.21,-4.37,39.87,33.76,76568.0,-8775.0,192045.0,-32.51,34.0,X-MC,3.72,57.0,-0.11,1.34,0.60,X40,ATH,FMCG
54,PAGEIND,51605.08,-0.22,-4.93,32.61,26.07,50761.0,-8080.0,155660.0,-29.94,39.0,X-MC,9.37,55.0,-0.16,1.08,0.99,X40,ATH,APPARELS
76,TMPV,600.00,-0.43,-15.24,67.74,42.18,158711.0,-42116.0,234294.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.49,XY24,NTT,AUTO
9,BAJAJHFL,181.50,1.56,-12.78,70.20,48.45,122773.0,-25616.0,174890.0,-19.01,43.0,X-MC,7.26,64.0,-0.21,1.22,1.60,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.39,-27.32,96.55,42.84,45613.0,-17763.0,47243.0,-52.62,42.0,X-SC,37.54,83.0,-0.39,0.33,2.80,XY24,NTT,MISC
60,RELAXO,1176.00,-0.41,-46.81,186.34,52.31,143877.0,-67948.0,77212.0,-43.37,38.0,X-SC,4.43,91.0,-0.47,0.54,3.35,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.18,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,25.0,X-SC,13.19,92.0,-0.50,0.58,0.18,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.26,-5.04,28.04,21.58,37314.0,-7065.0,133074.0,-52.09,38.0,X-MC,6.55,56.0,-0.19,0.93,17.81,X40N,ATH,IT
52,MEDANTA,1486.00,0.17,5.66,17.43,24.08,23379.0,7182.0,134132.0,-2.92,51.0,X-SC,5.52,89.0,0.31,0.93,26.38,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.40,-14.06,38.80,19.28,115357.0,-48643.0,297313.0,-25.24,63.0,X-LC,7.35,1.0,-0.42,2.07,9.91,X40,ATH,IT
41,INFY,2275.00,0.38,6.46,45.48,54.87,151489.0,20201.0,333089.0,-17.09,62.0,X-LC,2.62,2.0,0.13,2.32,15.27,X40,BTT,IT
76,TMPV,600.00,-0.43,-15.24,67.74,42.18,158711.0,-42116.0,234294.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.49,XY24,NTT,AUTO
82,VBL,671.64,-0.11,-6.07,44.44,35.67,131840.0,-19172.0,296670.0,-17.17,53.0,X-LC,5.77,4.0,-0.15,2.07,6.94,X40N,ATH,FMCG
43,ITC,452.00,0.09,-1.62,12.26,10.44,24139.0,-3242.0,196896.0,-41.15,43.0,X-LC,1.02,5.0,-0.13,1.37,3.85,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.85,-13.39,109.77,81.67,89184.0,-12564.0,81246.0,7305.56,50.0,L-SC,19.26,271.0,-0.14,0.57,57.34,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.18,-12.26,59.04,39.54,52404.0,-12399.0,88760.0,-31.43,66.0,M-SC,6.64,220.0,-0.24,0.62,25.16,AR,ATH,AUTO
51,MASFIN,398.61,-0.03,-5.88,29.67,22.05,27362.0,-5760.0,92220.0,-18.70,48.0,H-SC,6.99,164.0,-0.21,0.64,34.05,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.89,-18.99,51.88,23.03,75259.0,-34007.0,145064.0,57.89,50.0,H-SC,9.48,167.0,-0.45,1.01,43.93,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-2.65,-9.64,10.69,0.02,16861.0,-16831.0,157728.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.20,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.65,-9.64,10.69,0.02,16861.0,-16831.0,157728.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.20,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,1.18,1.64,18.99,20.95,48596.0,4136.0,255904.0,0.63,78.0,X-LC,14.61,31.0,0.09,1.78,37.86,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.09,-37.25,111.79,32.89,54156.0,-28762.0,48444.0,-698.49,63.0,L-MC,5.62,259.0,-0.53,0.34,33.41,XR,NTT,BANKS
0,5PAISA,593.00,1.16,-33.77,70.48,12.90,101989.0,-73790.0,144706.0,112.74,61.0,H-SC,13.32,161.0,-0.72,1.01,20.32,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.28,7.51,12.33,20.77,31039.0,17595.0,251739.0,-1.87,67.0,X-MC,3.79,62.0,0.57,1.75,14.95,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.77
1,25,44.64
2,50,75.52


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.53
MC    29.26
LC    25.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.89
X40      21.16
X40N     11.99
XR        9.51
XY25      9.09
AR        9.08
OX40N     7.68
X200      1.84
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.79
X-LC    20.03
M-SC    11.98
X-SC     8.23
H-MC     4.75
H-LC     2.99
L-SC     1.40
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.76,-4.63,38.47
IT,13.75,-14.50,73.61
FINANCE,9.48,-13.91,62.05
MISC,7.43,-21.79,71.69
ELECTRICAL,5.98,-9.26,49.29
PAINTS,5.97,-7.70,23.70
INSURANCE,4.43,0.59,34.44
PHARMA,3.87,-0.59,32.79
AUTO,3.37,-18.79,69.72


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3029401.0,21
XR,1295520.0,13
AR,1268839.0,10
X40,999507.0,14
X40N,727650.0,9
OX40N,688652.0,10
XY25,335419.0,6
SR,279677.0,2
MH,73446.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3542670.0,25
M-SC,1337470.0,15
X-MC,1209801.0,16
X-LC,886844.0,11
X-SC,736333.0,8
H-MC,395830.0,3
L-SC,260674.0,3
H-LC,129095.0,2
M-LC,118360.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213221.0      6
           AR         871025.0      5
           XR         790476.0      7
M-SC       XY24       769896.0      6
X-MC       X40        438105.0      7
X-LC       X40        387248.0      5
X-MC       XY24       357610.0      3
H-SC       OX40N      314825.0      4
X-SC       X40N       289794.0      3
M-SC       OX40N      286430.0      5
H-SC       SR         279677.0      2
X-SC       XY24       272385.0      3
X-MC       X40N       237031.0      4
X-LC       XY24       227306.0      2
H-MC       AR         206847.0      2
X-LC       X40N       200825.0      2
H-MC       XY24       188983.0      1
X-MC       XY25       177055.0      2
X-SC       X40        174154.0      2
L-SC       XR         173277.0      2
M-SC       XR         159251.0      2
           AR         121893.0      2
M-LC       XR         118360.0      1
L-SC       OX40N       87397.0      1
H-SC       MH          73446.0      1
X-LC       XY25        71465.0      2
H-LC       AR          69074.0      1
           X200        60021.0      1
L-MC       XR          54156.0      1
M-MC       XY25        46951.0      1
L-LC       XY25        39948.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
